<a href="https://colab.research.google.com/github/tonyw54/GeorgeBrown/blob/main/Fake_News_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2.2 Details of the project in ipython file

2.2.1 Defining the business problem

2.2.2 Defining clear description about dataset and the link to the dataset.

2.2.3 Define your approach

2.2.4 The type of code should be function based and modular AND Based on Object
Oriented.

2.2.4 Show and plot dataset and apply data analysis on Dataset

2.2.5 Explain detail about pipeline that you defined

2.2.6 show the process of training and plot training

2.2.7 Plot validation set and analyze the behavior of training and validation.

2.2.8 Write inference pipeline

2.2.9 Have session about lessons to learn for each member of the team by their own name.

2.2.1 Defining the business problem: Fake News Detection

A widely used Fake News classifier on HuggingFace is the hamzab/roberta-fake-news-classification model. This is model was created over two years ago and is now out of date.

2.2.2 Defining clear description about dataset and the link to the dataset

On Kaggel, there is a new, more up to date dataset at https://www.kaggle.com/datasets/evilspirit05/english-fake-news-dataset. This dataset was updated a few weeks ago, at the time of writing. This dataset addresses the business problem of **detecting and mitigating the spread of fake news**.

2.2.3 Define your approach

The HuggingFace hamzab/roberta-fake-news-classification model needs an upgrade. Using more modern data (recently published on Kaggel), we will update this model with further Fake News/Real News items.



In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from os import EX_PROTOCOL

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("hamzab/roberta-fake-news-classification")

model = AutoModelForSequenceClassification.from_pretrained("hamzab/roberta-fake-news-classification")


tokenizer_config.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
import torch
def predict_fake(title,text):
    input_str = "<title>" + title + "<content>" +  text + "<end>"
    input_ids = tokenizer.encode_plus(input_str, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
    device =  'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    with torch.no_grad():
        output = model(input_ids["input_ids"].to(device), attention_mask=input_ids["attention_mask"].to(device))
    return dict(zip(["Fake","Real"], [x.item() for x in list(torch.nn.Softmax()(output.logits)[0])] ))

In [ ]:
print(predict_fake("Donald Trump Kills Bigfoot","Bigfoot, the notorious secret head of the Democrat party, was shot by Donald Trump at Mar Lago this morning."))

{'Fake': 0.9999781847000122, 'Real': 2.176387897634413e-05}


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1553: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Here we see that this model does not do so well on current events.

In [ ]:
print(predict_fake("Donald Trump Refuses Third Debate","This morning, former President Donald Trump announced that he will not be participating in a new debate with Vice President Kamala Harris."))

{'Fake': 0.9999792575836182, 'Real': 2.079786099784542e-05}


In [ ]:
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

In [ ]:
!kaggle datasets download -d evilspirit05/english-fake-news-dataset
!unzip english-fake-news-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/evilspirit05/english-fake-news-dataset
License(s): MIT
 94% 9.00M/9.55M [00:00<00:00, 89.8MB/s]
100% 9.55M/9.55M [00:00<00:00, 93.6MB/s]
Archive:  english-fake-news-dataset.zip
  inflating: final_en.csv            


In [ ]:
DATASET_PATH = 'final_en.csv'  # Path to the dataset file
MAX_WORDS = 10000  # Maximum number of words to keep in the vocabulary
MAX_SEQUENCE_LENGTH = 100  # Maximum length of the sequences
EMBEDDING_DIM = 128  # Dimensionality of the word embeddings
LSTM_UNITS = 128  # Number of units in the LSTM layer
DROPOUT_RATE = 0.2  # Dropout rate for regularization
EPOCHS = 5  # Number of training epochs
BATCH_SIZE = 32  # Batch size for training

In [ ]:
# Load the data
def load_dataset(file_path):
    df = pd.read_csv(file_path)
    return df

# Preprocess the data
def preprocess_data(df):
  if 'lebel' in df.columns:
    df = df.rename(columns={'lebel': 'label'})
  df.fillna('', inplace=True)
  df['combined_text'] = df['title'] + ' ' + df['text']
  return df

# Convert into the model's expected format
def preprocess_function(examples):
    input_str = "<title>" + examples["title"] + "<content>" + examples["text"] + "<end>"
    model_inputs = tokenizer(input_str, padding="max_length", truncation=True)
    model_inputs["labels"] = examples["label"] # Add labels to the model inputs
    return model_inputs

In [ ]:
df = load_dataset(DATASET_PATH)
df = preprocess_data(df)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Reset indices after splitting
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_dataset = train_df.apply(lambda row: preprocess_function(row), axis=1)
test_dataset = test_df.apply(lambda row: preprocess_function(row), axis=1)

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.000000,0.005988


Epoch,Training Loss,Validation Loss
1,0.000000,0.005988
2,0.000000,0.006394
3,0.014800,0.005391


TrainOutput(global_step=3000, training_loss=0.00589430584354947, metrics={'train_runtime': 2516.5943, 'train_samples_per_second': 9.537, 'train_steps_per_second': 1.192, 'total_flos': 6314665328640000.0, 'train_loss': 0.00589430584354947, 'epoch': 3.0})

In [ ]:
# Save the trained model
#trainer.save_model("./trained_model")

# Save the tokenizer
#tokenizer.save_pretrained("./trained_model") # Add this line to save the tokenizer

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.json',
 './trained_model/merges.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [ ]:
from transformers import pipeline

# Load the trained model and tokenizer
loaded_model = AutoModelForSequenceClassification.from_pretrained("./trained_model")
loaded_tokenizer = AutoTokenizer.from_pretrained("./trained_model")

# Create a pipeline for text classification
classifier = pipeline("text-classification", model=loaded_model, tokenizer=loaded_tokenizer)

# Function to test the model
def test_model(title, text):
    input_str = "<title>" + title + "<content>" + text + "<end>"
    result = classifier(input_str)
    return result

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
title = "Donald Trump Kills Bigfoot"
text = "MIAMI (Reuters) - Bigfoot, the notorious secret head of the Democratic Party, was shot by Donald Trump at Mar Lago this morning."
prediction = test_model(title, text)
print(prediction)

[{'label': 'FAKE', 'score': 0.999995231628418}]


In [ ]:
title = "Former President Donald Trump celebrates birthday"
text = "MIAMI (Reuters) - On Monday, former United States President Donald Trump celebrated his seventy eighth birthday."
prediction = test_model(title, text)
print(prediction)

[{'label': 'TRUE', 'score': 0.9999934434890747}]


In [ ]:
title = "Russia warns Ukraine against use of long range missiles"
text = "MOSCOW (Reuters) - Russia on Wednesday warned Ukraine against striking targets inside Russia."
prediction = test_model(title, text)
print(prediction)

[{'label': 'TRUE', 'score': 0.9999934434890747}]


In [ ]:
title = "Israel and Hamas no closer to ceasefire deal"
text = "London (Reuters) - Israel and Hamas announced that they are still no closer to a deal to free Israeli hostages and end the current war in Gaza."
prediction = test_model(title, text)
print(prediction)

[{'label': 'TRUE', 'score': 0.9999934434890747}]


In [ ]:
# Save model to google drive

#from google.colab import drive
#drive.mount('/content/drive')
#!cp -r trained_model /content/drive/MyDrive/

Mounted at /content/drive


In [ ]:
# Load trained model from google drive
#from google.colab import drive
#drive.mount('/content/drive')

#loaded_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/trained_model")
#loaded_tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/trained_model")